<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl_2/blob/main/Biomate_NN_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import matplotlib.pyplot as plt
import tqdm
from time import time

# Estructura de los datos.

Consideramos un caso general, una base con datos de frecuencias de especies, cada observación de dimensión $k$, y supongamos que tenemos $n$ observaciones.

El output busca ser la clasificación en una de $d$ categorías, definidas ya sea por ciudad o localización-clima (pero de un sólo tipo).


# Datos de ejemplo

In [160]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Dataset_w_tags.csv")
df.head()
from sklearn.utils import shuffle
df = shuffle(df)

In [161]:
print(df.shape)

(237, 513)


Se tienen 237 observaciones, cada una es una muestra en una "localización".
Por renglón, se tienen frecuencias de "bichos" de 510 "dominant taxa".

En este caso, $k=510$, $n=237$.

In [162]:
ytags=df['Tag']
print(ytags)

234         Cold forests_3
230    Temperate forests_0
21                Boreal_9
228          Dry forests_0
137          Dry forests_0
              ...         
218    Temperate forests_0
46       Cold grasslands_1
131           Grasslands_2
151      Cold grasslands_5
41       Cold grasslands_1
Name: Tag, Length: 237, dtype: object


In [163]:
import numpy as np
_,idx = np.unique(ytags,return_inverse=True)

Dados los datos con su respectiva clasificación. (En el ejemplo $d=12$)

# Definición del modelo.

# Partición de datos

In [164]:
from keras.utils import to_categorical
X_total=df.drop(columns=['Tag','Dominant_taxa_ID/ID_Environmental'])
Y_total=to_categorical(idx)

In [165]:
X_train=X_total[0:200]
Y_train=Y_total[0:200]

In [166]:
df.head()
k,n=X_train.shape
n=n#-indice, -tag
print(k,n)

200 511


In [167]:
input_shape = (n,)#=n
output_shape=len(np.unique(ytags))

In [168]:
import tensorflow as tf
import keras
import pandas as pd
from keras import layers
from keras import models


In [169]:
model = models.Sequential()

model.add(layers.InputLayer(input_shape=(n,), name='Input_Layer'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(37, activation='Softmax', name='Output_Layer'))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
import time
tic = time.time()

model.fit(x = X_train, 
           y = Y_train, 
          validation_data=[X_total[200:], Y_total[200:]],
          batch_size=50,
           epochs=20,
           verbose=2,shuffle=True)

print('seconds=', time.time()-tic)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 64)                32768     
                                                                 
 dense_47 (Dense)            (None, 32)                2080      
                                                                 
 Output_Layer (Dense)        (None, 37)                1221      
                                                                 
Total params: 36,069
Trainable params: 36,069
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
4/4 - 0s - loss: 21.5206 - accuracy: 0.0400 - val_loss: 14.0334 - val_accuracy: 0.1351 - 403ms/epoch - 101ms/step
Epoch 2/20
4/4 - 0s - loss: 10.2344 - accuracy: 0.1800 - val_loss: 10.6137 - val_accuracy: 0.2162 - 23ms/epoch - 6ms/step
Epoch 3/20
4/4 - 0s - loss: 6.8388 - accuracy: 0.2800 - val_loss: 